In [ ]:
# ==========================================
# 1. ADIM: KURULUM VE DRİVE BAĞLANTISI
# ==========================================
from google.colab import drive
import os

# Drive Bağla
drive.mount('/content/drive')

# Gerekli kütüphaneyi kur (MIT-BIH okumak için)
!pip install -q wfdb

import wfdb
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, Input
from sklearn.model_selection import train_test_split
from sklearn.utils import resample

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 141.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.3.3 which is incompatible.


In [ ]:
# ==========================================
# 2. ADIM: VERİ İNDİRME (Senin düzenin)
# ==========================================
# Veriyi senin klasörüne indiriyoruz (zaten varsa indirmez)
data_dir = "/content/drive/MyDrive/mitdb"
os.makedirs(data_dir, exist_ok=True)

# Veri setini kontrol et, yoksa indir
if len(os.listdir(data_dir)) < 10:
    print("MIT-BIH veriseti indiriliyor...")
    !wget -q -r -N -c -np https://physionet.org/files/mitdb/1.0.0/ -P /content/
    !cp -r /content/physionet.org/files/mitdb/1.0.0/* "{data_dir}/"
    print("İndirme tamamlandı.")
else:
    print("MIT-BIH veriseti zaten Drive'da mevcut.")

MIT-BIH veriseti zaten Drive'da mevcut.


In [ ]:
# ==========================================
# 3. ADIM: RR ARALIKLARINI ÇIKARMA (ÖNEMLİ KISIM)
# ==========================================
# Hangi sembollerin "Normal", hangilerinin "Aritmi" olduğunu tanımlıyoruz
# N: Normal, L: Left bundle branch block, R: Right... (Bunları Normal sayabiliriz veya ayırabiliriz)
# Basitlik için: Normal dışı her şeye "1" (Aritmi) diyeceğiz.

NORMAL_SYMBOLS = ['N', 'L', 'R', 'e', 'j']
# Diğer tüm semboller Aritmi (A, V, F, / ...) kabul edilecek

def load_data_from_mitbih(records, seq_len=10):
    X_all = []
    y_all = []

    print("Veriler işleniyor (RR Aralıkları hesaplanıyor)...")

    for record in records:
        try:
            # Sadece anotasyon dosyasını (ATR) okumak yeterli, sinyale gerek yok!
            # Çünkü bize sadece "vuruş zamanları" lazım.
            ann = wfdb.rdann(f"{data_dir}/{record}", 'atr')

            # RR Aralıklarını Hesapla (Sample farkı / 360Hz)
            # np.diff: Bir sonraki vuruştan öncekini çıkarır
            rr_intervals = np.diff(ann.sample) / 360.0

            # Sembolleri al (İlk vuruşun RR'ı olmadığı için sembollerde 1. indexten başlarız)
            symbols = np.array(ann.symbol[1:])

            # RR ve Sembol uzunluğunu eşitle
            min_len = min(len(rr_intervals), len(symbols))
            rr_intervals = rr_intervals[:min_len]
            symbols = symbols[:min_len]

            # --- VALIDASYON ---
            # Çok uzun (örn > 2sn) veya çok kısa (örn < 0.2sn) hatalı ölçümleri atabiliriz
            valid_indices = (rr_intervals > 0.2) & (rr_intervals < 2.0)
            rr_intervals = rr_intervals[valid_indices]
            symbols = symbols[valid_indices]

            # --- SEQUENCE OLUŞTURMA (LSTM İÇİN) ---
            # Son 10 vuruşa bakıp, sıradaki vuruşun ne olduğunu tahmin edeceğiz (veya o dizinin durumunu)
            for i in range(len(rr_intervals) - seq_len):
                sequence = rr_intervals[i : i+seq_len]
                target_symbol = symbols[i+seq_len] # Sequence sonundaki vuruş

                # Etiketleme: Normal mi Aritmi mi?
                label = 0 if target_symbol in NORMAL_SYMBOLS else 1

                # Sadece 'bilinen' beat tiplerini al (Gürültü vs. atla)
                if target_symbol in NORMAL_SYMBOLS or target_symbol in ['A', 'a', 'J', 'S', 'V', 'E', 'F']:
                    X_all.append(sequence)
                    y_all.append(label)

        except Exception as e:
            continue

    return np.array(X_all), np.array(y_all)

# Kayıt Listesi (100 ile 234 arasındaki dosyalar)
all_records = [f.replace('.dat','') for f in os.listdir(data_dir) if f.endswith('.dat')]
# Hepsini kullanmak uzun sürebilir, demo için ilk 10-20 tanesini alalım
# Hepsini istersen: selected_records = all_records
selected_records = all_records[:20]

X, y = load_data_from_mitbih(selected_records, seq_len=10)

print(f"\nToplam Örnek Sayısı: {len(X)}")
print(f"Normal Sayısı: {sum(y==0)}")
print(f"Aritmi Sayısı: {sum(y==1)}")

# --- DENGESİZ VERİ SETİ DÜZELTME (Opsiyonel ama Önerilir) ---
# Normal veri çok fazladır, Aritmi azdır. Eşitleyelim.
X_normal = X[y==0]
X_arrhythmia = X[y==1]

# Normalleri azalt (Undersampling)
X_normal_down = resample(X_normal, replace=False, n_samples=len(X_arrhythmia), random_state=42)

X_balanced = np.concatenate([X_normal_down, X_arrhythmia])
y_balanced = np.concatenate([np.zeros(len(X_arrhythmia)), np.ones(len(X_arrhythmia))])

print(f"Dengelenmiş Veri Seti: {len(X_balanced)} örnek")

# Eğitim/Test Ayırma
X_train, X_test, y_train, y_test = train_test_split(X_balanced, y_balanced, test_size=0.2, random_state=42)

# LSTM Formatına Sokma: [Örnek, Zaman, Özellik]
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

Veriler işleniyor (RR Aralıkları hesaplanıyor)...

Toplam Örnek Sayısı: 35599
Normal Sayısı: 34139
Aritmi Sayısı: 1460
Dengelenmiş Veri Seti: 2920 örnek


In [ ]:
# ==========================================
# 4. ADIM: LSTM MODEL EĞİTİMİ
# ==========================================
model = Sequential()
model.add(Input(shape=(10, 1))) # 10 adım geriye bakıyoruz
model.add(LSTM(64, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid')) # 0 veya 1

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

print("\nModel Eğitiliyor...")
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))


Model Eğitiliyor...
Epoch 1/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - accuracy: 0.5060 - loss: 0.6942 - val_accuracy: 0.4983 - val_loss: 0.6923
Epoch 2/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5241 - loss: 0.6924 - val_accuracy: 0.5325 - val_loss: 0.6910
Epoch 3/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5093 - loss: 0.6912 - val_accuracy: 0.5017 - val_loss: 0.6930
Epoch 4/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5091 - loss: 0.6919 - val_accuracy: 0.5342 - val_loss: 0.6861
Epoch 5/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.5413 - loss: 0.6849 - val_accuracy: 0.5034 - val_loss: 0.6903
Epoch 6/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5439 - loss: 0.6867 - val_accuracy: 0.5394 - val_loss: 0.6761
Epoch 7/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5497 - loss: 0.6759 - val_accuracy: 0.5565 - val_loss: 0.6743
Epoch 8/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5661 - loss: 0.6722 - val_accura

In [ ]:
# ==========================================
# 5. ADIM: MODELİ KAYDETME
# ==========================================
save_path = '/content/drive/MyDrive/mitdb/lstm_arrhythmia_model.h5'
model.save(save_path)
print(f"\n✅ Model kaydedildi: {save_path}")
print("Bu dosyayı indirip Raspberry Pi'ye atabilirsin.")


✅ Model kaydedildi: /content/drive/MyDrive/mitdb/lstm_arrhythmia_model.h5
Bu dosyayı indirip Raspberry Pi'ye atabilirsin.


In [ ]:
from tensorflow.keras.models import load_model

save_path = '/content/drive/MyDrive/mitdb/lstm_arrhythmia_model.h5'
model = load_model(save_path)
print(f"Model başarıyla yüklendi: {save_path}")

Model başarıyla yüklendi: /content/drive/MyDrive/mitdb/lstm_arrhythmia_model.h5


In [ ]:
from sklearn.metrics import classification_report

# Test seti üzerinde tahminler yap
y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int) # Eşik değeri 0.5 ile ikili tahminlere çevir

# Sınıflandırma raporunu yazdır
print("\nModel Performans Raporu:")
print(classification_report(y_test, y_pred, target_names=['Normal', 'Aritmi']))

19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step

Model Performans Raporu:
              precision    recall  f1-score   support

      Normal       0.71      0.92      0.81       291
      Aritmi       0.89      0.63      0.74       293

    accuracy                           0.78       584
   macro avg       0.80      0.78      0.77       584
weighted avg       0.80      0.78      0.77       584

